# nb1: Genealogical variation and sequence variation


### Outline for notebooks (1-3)
1. A visual introduction to genealogical variation. 
2. Connecting genealogies to species trees (demographic model).
3. Connecting genealogies to sequence variation (observations)
4. Connecting genealogies to gene trees (inference)

### Learning objectives: 
By the end of this notebook series you should:
1. Be familiar with the `toytree` and `ipcoal` Python libraries.
2. Recognize the power of coalescent simulations to test hypotheses.
3. Have an improved understanding of gene-tree/species-tree concepts.


### Additional recommended reading:

- [Rosenberg and Nordborg (2002) Genealogical trees, coalescent theory and the analysis of genetic polymorphisms. Nature Review Genetics](https://eaton-lab.org/slides/genomics/readings/Rosenberg-and-Nordborg-2002.pdf)

### The toytree and ipcoal packages
The [toytree](https://toytree.readthedocs.io) and [ipcoal](https://ipcoal.readthedocs.io) Python packages are designed to be used together within jupyter notebooks to execute interactive code to create, manipulate, and visualize tree data objects, and to simulate the coalescent process and generate sequence data. The `ipcoal` package is built as a wrapper around the popular `msprime` coalescent simulator, and extends the functionality of this package for phylogenetic-scale analyses. 

In [ ]:
import ipcoal
import toytree

### Start with a species tree
In this example we will start with a known parameterized species tree model from which we will sample genealogical histories. Parameters of this model, such as the effective population sizes (Ne), will affect the amount of genealogical variation. A species tree model defines a topology (branching order) with edge lengths (in units of generations), and effective population sizes on each edge. This effectively defines a container in which genealogies evolve. Let's start by using toytree to generate a random species tree topology with a given crown height (1M generations), and visualize the tree.

In [ ]:
# generate a random species tree with 10 tips and a crown age of 10M generations
tree = toytree.rtree.unittree(10, treeheight=1e6, seed=123)

# draw the species tree
tree.draw(ts='c', tip_labels=True);

To sample a genealogy from the species tree above we will create an `ipcoal.Model` object by providing it the species tree object as a parameter, and an Ne value as an additional parameter. Here the Ne value is relatively low compared to the edge lengths and thus discordance on the species tree is low. The genealogy (shown with orange edges) matches the species tree topology (shown with black edges above), meaning that there is no discordance in this example. 

In [ ]:
# setup a coalescent simulator for this species tree and Ne value
model1 = ipcoal.Model(tree=tree, Ne=1e4)

# sample a genealogy from this model
model1.sim_trees(1)

# draw the genealogy
model1.draw_genealogy(idx=0, edge_colors='orange');

Another way to visualize this is to examine coalescent times with respect to the divergence/speciation times of the lineages. Here you can see that coalescent events occur almost instantaneously within each edge of the species tree. Each sample is colored differently going backwards in time from the tips of the species tree until they coalesce with another sample. 

In [ ]:
model1.draw_demography(idx=0, spacer=1, height=300);

Now let's look at what happens when the Ne value is increased. Here we set Ne to 2e5 across the entire tree, representing a higher level of expected discordance. The sampled genealogy below no longer matches the species tree topology. And similarly, when we examine the coalescent times with respect to the species tree divergences in the next plot, we can see that many deep coalescent events occured near the root of the species tree. When this occurs the relationships among samples in the genealogy are no longer required to match the species tree. Try executing the cell below multiple times to examine stochastic coalescent variation over multiple sampled genomic regions. 

In [ ]:
# sample a genealogy from this species tree with Ne=2e5
model2 = ipcoal.Model(tree=tree, Ne=2e5)
model2.sim_trees(1)

# draw the genealogy
model2.draw_genealogy(idx=0, edge_colors='orange');

# draw the genealogy within a container
model2.draw_demography(idx=0, spacer=1, height=300);

### Setting and checking demographic parameters
In addition to setting Ne as a fixed value across the tree, we can also set variable Ne values across different edges of the tree. The best way to accomplish this is by using toytree to set values to directly on nodes of the tree. This allows us to visually assess that the simulation was setup properly. Here I use the `.set_node_values` function of the toytree object to set greater Ne values on several nodes by referencing their node index numbers. This returns a new edited copy of the tree object with I store as a variable named `vtree` (variable Ne tree). 


I then draw the vtree object using the built-in option `treestyle='p'`, which shows the node index numbers on each node, and automatically displays Ne values from nodes as edge widths to represent differences in effective population sizes.


In [ ]:
# create a new tree copy with Ne values mapped to nodes
vtree = tree.set_node_values(
    feature="Ne",
    values={i: 2e5 for i in (6, 7, 8, 9, 12, 15, 17)},
    default=1e4,
)

# draw with ts='p' to show Ne on edges
vtree.draw(ts='p');

Again, we can sample a single genealogy from this species tree model and compare it to the species tree. The container plot (`draw_demography`) shows the genealogy within the species tree. The coalescent times of the genealogy are always necessarily deeper than the splits in the species tree, since the species tree container sets limits on when coalescent events can occur. In this example, where `vtree` has variable Ne values, it is clear that coalescent events occur much more quickly in the subclade with small Ne values than in the other subclade with larger Ne. 

In [ ]:
# create simulator 
model3 = ipcoal.Model(tree=vtree)

# sample 1 genealogy
model3.sim_trees(1)

# draw genealogy alone to show discordance
model3.draw_genealogy(idx=0, edge_colors='orange');

# draw genealogy within container 
model3.draw_demography(idx=0, spacer=1, height=300);

So far we have been sampling only a single genealogy at a time, but in most cases we are more interested in the **distribution of coalescent variation** over many sampled genealogies. It is simple to sample a large number of genealogies using the `sim_trees` function from an `ipcoal.Model` object. Here we sample 100 genealogies and use the multitree plotting functions from toytree to visualize the variation among genealogies as a cloud of overlapping trees.

In [ ]:
# sample 100 genealogies
model3.sim_trees(100)

# draw a cloud tree of 100 samples
mtre = toytree.mtree(model3.df.genealogy)
mtre.draw_cloud_tree(layout='d', edge_colors='orange');

### Genealogical variation and sequence variation
Across a genome alignment different regions will trace back different coalescent histories, representing the fact that each genome is a mosaic of many different ancestors due to the effects of recombination. Using `ipcoal`, we can examine this variation in a number of ways. Let's first examine *unlinked* variation, representing genealogies that are *statistically independent* of each other. These can be sampled by using the functions `sim_trees`, `sim_loci`, or `sim_snps`. The latter two functions simulate sequence data on the sampled genealogies, and can do so using complex finite substitution models (e.g., JC, HKY). 

In [ ]:
# sample 10 unlinked genealogies
model = ipcoal.Model(tree, Ne=2e5, seed=666)
model.sim_trees(10)

In [ ]:
# draw several unlinked genealogies
mtre = toytree.mtree(model.df.genealogy)
mtre.draw(ts='c', layout='d', tip_labels=True, shared_axes=True, height=200);

The four genealogies above represent four random samples from the distribution of genealogical variation that exists given the defined species tree. As you can see, the topology and edge lengths vary considerably among the set of trees, representing a high level of discordance. This *discordance* can be visualized more clearly by using the `fixed_order=True` option, like in the plot below. Here the same exact four trees are show, but with the tip order fixed across each drawing so that discordance is easier to see. 

In [ ]:
# same drawing but with fixed tip order to highlight discordance
mtre.draw(ts='c', layout='d', tip_labels=True, shared_axes=True, height=200, fixed_order=True);

### Simulate sequence data
Sequence data can be simulated on one or more genealogies to represent linked or unlinked sequence variation. Simulated sequences can also be visualized, manipulated, and exported to a number of file formats. 

`sim_snps` will continue to run simulations until it produces the requested number of variable sites. Each SNP is statistically unlinked from the others. 

`sim_loci` returns the number of requested sites whether or not they accumulate substitutions during the simulation process. The sites within a locus are in linkage disequilibrium, meaning they are statistically non-indepedent, or linked. 

In [ ]:
# sample sequence data on genealogies
model.sim_snps(10)

# visualize the sequence variation
model.draw_seqview(show_text=True);

In [ ]:
# sample sequence data on genealogies
model.sim_loci(nloci=1, nsites=30)

# visualize the sequence variation
model.draw_seqview(show_text=True);

### Simulating linked data
In some cases we may be interested in simulating the effect of genetic linkage. For example, to model the spatial variation in genealogies (and sequences) along a chromosome. Or to properly model the expected variation within large loci (e.g., 10Kb) that could arise from recombination. Let's examine that now. This simulation is similar to the one above where we sampled 10 unlinked genealogies, but here we sample multiple linked genealogies. You can see in the plot below that each genealogy differs slightly from the one that comes before it, perhaps only in branch lengths. This is the effect of genetic linkage. The genealogies in neighboring genomic blocks share many of the same genealogical ancestors, and thus are highly correlated. 

In [ ]:
# sample 10 unlinked genealogies
model = ipcoal.Model(tree, Ne=2e5, recomb=1e-9)

# sample trees from 1 10Kb locus
model.sim_trees(nloci=1, nsites=10000)

# draw several unlinked genealogies
mtre = toytree.mtree(model.df.genealogy)

# draw linked genealogies with fixed order
mtre.draw(ts='c', layout='d', tip_labels=True, shared_axes=True, height=200, fixed_order=True);

### Simulate sequence data
We can use this framework to simulate more realistic sequence data that approximates the type of data that is produced by existing genome sequencing technologies. This could include many short loci which would look similar to RAD-seq, or a small number of longer loci, approximating hybrid capture probes. Each locus is treated as independent of other loci, but sites within a locus are linked, with the amount of linkage dependent on model parameters (affected by `recomb`, `Ne`, and edge lengths of the `tree` topology). The default recomb rate was defined in our current model, which is 1e-9.  

In [ ]:
# generate sequence data for n loci each m bp in length
model.sim_loci(nloci=100, nsites=150)

Model objects will store a summary of simulation results in the `.df` attribute. In the example below you can see loci that are represented by multiple genealogies, each covering some part of its extent. When sequence data is simulated on a locus represented by multiple genealogies the resulting sequence data represents the concatenation of mutations evolved on more than one genealogical history. This can introduce error into gene tree estimation if loci are very long, since the data are not derived from a single genealogical history. 

In [ ]:
# the genealogical variation is also stored in a dataframe
model.df.head(10)

The sequence data can also be accessed directly as an integer numpy array. See the ipcoal documentation for more information about ways to interact with this sequence matrix. Here we will not go into detail, but only visualize and export sequence data (many options are available for this).

In [ ]:
# the size of the sequence matrix
model.seqs.shape

In [ ]:
# view part of the first locus
model.draw_seqview(idx=0, start=0, end=50);

In [ ]:
# write the loci as a concatenated alignment
model.write_concat_to_phylip(name="test", outdir="/tmp")

In [ ]:
# write the first 10 loci as a individual phylip files
model.write_loci_to_phylip(outdir="/tmp", idxs=range(10))

### Migration (gene flow)
You can also easily visualize and model migration (gene flow) in coalescent simulations with ipcoal and toytree. This can be used to validate methods for detecting hybrid introgression (e.g., make sure your migration flows the right way!), or to examine its effect on other phylogenetic inference methods. To draw admixture edges you only need to designate the source and destination node indices in toytree. To designate admixture in an ipcoal simulation you must provide (source, destination, timing, proportion), where the timing of introgression is entered as a proportion of the shared edge length between two edges (e.g., 0.5)).

In [ ]:
# draw a hybrid edge on a tree by designatin the source and destination node indices
tree.draw(ts='p', admixture_edges=(5, 6));

Here we simulate genealogies on the phylogenetic network above using relatively small effective population sizes such that there is very little discordance except that which is caused by the additional admixture edge, which allows these two divergent samples to occasionally experience rapid coalesce.

In [ ]:
# simulate unlinked data with admixture
model4 = ipcoal.Model(tree, Ne=2e4, admixture_edges=[(5, 6, 0.5, 0.25)])

# sample genealogies from a species network
model4.sim_trees(200)

# draw a cloud tree of 100 samples
mtre = toytree.mtree(model4.df.genealogy)
mtre.draw_cloud_tree(
    layout='d',
    edge_style={"stroke-opacity": 0.01},
    fixed_order=tree.get_tip_labels(),
);

When we increase the effective population size such that discordance is caused by both incomplete lineage sorting and by admixture you can see that the admixture signal is less clear. In this case, to distinguish these two sources of discordance from each other may require a larger number of sampled loci. This is a demonstration of why sampling thousands of loci from across the genome is sometimes required to test evolutionary hypotheses.

In [ ]:
# simulate unlinked data with admixture
model4 = ipcoal.Model(tree, Ne=1e5, admixture_edges=[(5, 6, 0.5, 0.25)])

# sample genealogies from a species network
model4.sim_trees(200)

# draw a cloud tree of 100 samples
mtre = toytree.mtree(model4.df.genealogy)
mtre.draw_cloud_tree(
    layout='d',
    edge_style={"stroke-opacity": 0.01},
    fixed_order=tree.get_tip_labels(),
    height=400,
);